In [1]:
import os
import glob
import pandas as pd
#os.chdir("s3://bigdata-group3//df2018Q2")

In [2]:
# Start SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("group3_project").getOrCreate()

In [3]:
# Start Spark Context
from pyspark import SparkContext, SparkConf
sc = spark.sparkContext

In [4]:
spark

In [5]:
sc

<SparkContext master=yarn appName=group3_project>

In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer,IndexToString, VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions

In [10]:
df = spark.read\
  .format('parquet')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdata-group3/mergedata/hardrive.parquet/*.parquet')

In [11]:
df.count()

76714860

In [8]:
mydf = spark.read.parquet('s3://bigdata-group3/mergedata/hardrive.parquet/*.parquet')

In [12]:
mydf.count()

76714860

In [9]:
mydf.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: double (nullable = true)
 |-- smart_1_raw: double (nullable = true)
 |-- smart_2_normalized: double (nullable = true)
 |-- smart_2_raw: double (nullable = true)
 |-- smart_3_normalized: double (nullable = true)
 |-- smart_3_raw: double (nullable = true)
 |-- smart_4_normalized: double (nullable = true)
 |-- smart_4_raw: double (nullable = true)
 |-- smart_5_normalized: double (nullable = true)
 |-- smart_5_raw: double (nullable = true)
 |-- smart_7_normalized: double (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_8_normalized: double (nullable = true)
 |-- smart_8_raw: double (nullable = true)
 |-- smart_9_normalized: double (nullable = true)
 |-- smart_9_raw: double (nullable = true)
 |-- smart_10_normalized: double (nullable = t

In [13]:
# Count the NA for each columns
type(mydf.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in mydf.columns)).show())

+----+-------------+-----+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---------------

NoneType

In [14]:
drop_columns=[
    'smart_2_normalized',
    'smart_2_raw',
    'smart_8_normalized',
    'smart_8_raw',
    'smart_11_normalized',
    'smart_11_raw',
    'smart_13_normalized',
    'smart_13_raw',
    'smart_15_normalized',
    'smart_15_raw',
    'smart_22_normalized',
    'smart_22_raw',
    'smart_177_normalized',
    'smart_177_raw',
    'smart_179_normalized',
    'smart_179_raw',
    'smart_181_normalized',
    'smart_181_raw',
    'smart_182_normalized',
    'smart_182_raw',
    'smart_183_normalized',
    'smart_183_raw',
    'smart_184_normalized',
    'smart_184_raw',
    'smart_189_normalized',
    'smart_189_raw',
    'smart_191_normalized',
    'smart_191_raw',
    'smart_195_normalized',
    'smart_195_raw',
    'smart_196_normalized',
    'smart_196_raw',
    'smart_200_normalized',
    'smart_200_raw',
    'smart_201_normalized',
    'smart_201_raw',
    'smart_220_normalized',
    'smart_220_raw',
    'smart_222_normalized',
    'smart_222_raw',
    'smart_223_normalized',
    'smart_223_raw',
    'smart_224_normalized',
    'smart_224_raw',
    'smart_225_normalized',
    'smart_225_raw',
    'smart_226_normalized',
    'smart_226_raw',
    'smart_235_normalized',
    'smart_235_raw',
    'smart_250_normalized',
    'smart_250_raw',
    'smart_251_normalized',
    'smart_251_raw',
    'smart_252_normalized',
    'smart_252_raw',
    'smart_254_normalized',
    'smart_254_raw',   
    'smart_255_normalized',
    'smart_255_raw'
]
mydf_new=mydf.drop(*drop_columns)

In [15]:
mydf_new.columns

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_normalized',
 'smart_197_raw',
 'smart_198_normalized',
 'smart_198_raw',
 'smart_199_normalized',
 'smart_199_raw',
 'smart_240_normalized',
 'smart_240_raw',
 'smart_241_normalized',
 'smart_241_raw',
 'smart_242_normalized',
 'smart_242_raw']

In [16]:
mydf_new.count()

76714860

In [17]:
mydf.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: double (nullable = true)
 |-- smart_1_raw: double (nullable = true)
 |-- smart_2_normalized: double (nullable = true)
 |-- smart_2_raw: double (nullable = true)
 |-- smart_3_normalized: double (nullable = true)
 |-- smart_3_raw: double (nullable = true)
 |-- smart_4_normalized: double (nullable = true)
 |-- smart_4_raw: double (nullable = true)
 |-- smart_5_normalized: double (nullable = true)
 |-- smart_5_raw: double (nullable = true)
 |-- smart_7_normalized: double (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_8_normalized: double (nullable = true)
 |-- smart_8_raw: double (nullable = true)
 |-- smart_9_normalized: double (nullable = true)
 |-- smart_9_raw: double (nullable = true)
 |-- smart_10_normalized: double (nullable = t

## Model

In [22]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, IndexToString, VectorAssembler, Binarizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [144]:
mydf = spark.read.parquet('s3://bigdata-group3/mergedata/hardrive.parquet/*.parquet')

In [145]:
drop_columns=[
    'smart_2_normalized',
    'smart_2_raw',
    'smart_8_normalized',
    'smart_8_raw',
    'smart_11_normalized',
    'smart_11_raw',
    'smart_13_normalized',
    'smart_13_raw',
    'smart_15_normalized',
    'smart_15_raw',
    'smart_22_normalized',
    'smart_22_raw',
    'smart_177_normalized',
    'smart_177_raw',
    'smart_179_normalized',
    'smart_179_raw',
    'smart_181_normalized',
    'smart_181_raw',
    'smart_182_normalized',
    'smart_182_raw',
    'smart_183_normalized',
    'smart_183_raw',
    'smart_184_normalized',
    'smart_184_raw',
    'smart_189_normalized',
    'smart_189_raw',
    'smart_191_normalized',
    'smart_191_raw',
    'smart_195_normalized',
    'smart_195_raw',
    'smart_196_normalized',
    'smart_196_raw',
    'smart_200_normalized',
    'smart_200_raw',
    'smart_201_normalized',
    'smart_201_raw',
    'smart_220_normalized',
    'smart_220_raw',
    'smart_222_normalized',
    'smart_222_raw',
    'smart_223_normalized',
    'smart_223_raw',
    'smart_224_normalized',
    'smart_224_raw',
    'smart_225_normalized',
    'smart_225_raw',
    'smart_226_normalized',
    'smart_226_raw',
    'smart_235_normalized',
    'smart_235_raw',
    'smart_250_normalized',
    'smart_250_raw',
    'smart_251_normalized',
    'smart_251_raw',
    'smart_252_normalized',
    'smart_252_raw',
    'smart_254_normalized',
    'smart_254_raw',   
    'smart_255_normalized',
    'smart_255_raw'
]
mydf_new=mydf.drop(*drop_columns)

In [146]:
mydf_new.columns

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_normalized',
 'smart_197_raw',
 'smart_198_normalized',
 'smart_198_raw',
 'smart_199_normalized',
 'smart_199_raw',
 'smart_240_normalized',
 'smart_240_raw',
 'smart_241_normalized',
 'smart_241_raw',
 'smart_242_normalized',
 'smart_242_raw']

In [61]:
#mydf_new.count()

76714860

In [147]:
mydf_new=mydf_new.na.drop()

In [148]:
#mydf_new.show(10)

In [124]:
#mydf_new.count()

57364274

In [125]:
colname=mydf_new.columns

In [126]:
colname

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_normalized',
 'smart_197_raw',
 'smart_198_normalized',
 'smart_198_raw',
 'smart_199_normalized',
 'smart_199_raw',
 'smart_240_normalized',
 'smart_240_raw',
 'smart_241_normalized',
 'smart_241_raw',
 'smart_242_normalized',
 'smart_242_raw']

In [127]:
colname.remove('date')

In [128]:
colname.remove('failure')

In [129]:
colname.remove('model')

In [130]:
colname.remove('serial_number')

In [131]:
mydf_new.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: double (nullable = true)
 |-- smart_1_raw: double (nullable = true)
 |-- smart_3_normalized: double (nullable = true)
 |-- smart_3_raw: double (nullable = true)
 |-- smart_4_normalized: double (nullable = true)
 |-- smart_4_raw: double (nullable = true)
 |-- smart_5_normalized: double (nullable = true)
 |-- smart_5_raw: double (nullable = true)
 |-- smart_7_normalized: double (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_9_normalized: double (nullable = true)
 |-- smart_9_raw: double (nullable = true)
 |-- smart_10_normalized: double (nullable = true)
 |-- smart_10_raw: double (nullable = true)
 |-- smart_12_normalized: double (nullable = true)
 |-- smart_12_raw: double (nullable = true)
 |-- smart_187_normalized: double (nullabl

In [132]:
modelcol = StringIndexer(inputCol="model", outputCol="model_ix").setHandleInvalid("skip")
serial_number = StringIndexer(inputCol="serial_number", outputCol="serial_number_ix").setHandleInvalid("skip")

In [133]:
colname.insert(0,"model_ix")   

In [134]:
colname.insert(0,"serial_number_ix") 

In [135]:
colname

['serial_number_ix',
 'model_ix',
 'capacity_bytes',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_188_normalized',
 'smart_188_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_192_normalized',
 'smart_192_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_normalized',
 'smart_197_raw',
 'smart_198_normalized',
 'smart_198_raw',
 'smart_199_normalized',
 'smart_199_raw',
 'smart_240_normalized',
 'smart_240_raw',
 'smart_241_normalized',
 'smart_241_raw',
 'smart_242_normalized',
 'smart_242_raw']

In [136]:
vectorAssembler_features = VectorAssembler(
    inputCols=colname, 
    outputCol="features")

In [137]:
#help_binarizer = Binarizer(threshold=0, inputCol="failure", outputCol="label") 
#mydf_new = help_binarizer.transform(mydf_new)

In [138]:
splitted_data = mydf_new.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 45892843
Number of testing records : 10324465
Number of prediction records : 1146966


In [142]:
train_data.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: double (nullable = true)
 |-- smart_1_raw: double (nullable = true)
 |-- smart_3_normalized: double (nullable = true)
 |-- smart_3_raw: double (nullable = true)
 |-- smart_4_normalized: double (nullable = true)
 |-- smart_4_raw: double (nullable = true)
 |-- smart_5_normalized: double (nullable = true)
 |-- smart_5_raw: double (nullable = true)
 |-- smart_7_normalized: double (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_9_normalized: double (nullable = true)
 |-- smart_9_raw: double (nullable = true)
 |-- smart_10_normalized: double (nullable = true)
 |-- smart_10_raw: double (nullable = true)
 |-- smart_12_normalized: double (nullable = true)
 |-- smart_12_raw: double (nullable = true)
 |-- smart_187_normalized: double (nullabl

In [83]:
#from pyspark.ml.classification import RandomForestClassifier
#rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

In [116]:
import time

## Logistic

In [84]:
log = LogisticRegression(labelCol="failure", featuresCol="features",maxIter=10, regParam=0.01)

In [85]:
pipeline_log = Pipeline(stages=[modelcol, serial_number,
                               vectorAssembler_features, 
                               log])

In [86]:
model_log = pipeline_log.fit(train_data)

In [96]:
predictions_log.count()

10324461

In [113]:
predictions_log = model_log.transform(test_data)
evaluatorLog = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("failure")
accuracy_log = evaluatorLog.evaluate(predictions_log)
print("Logistic Regression Accuracy = %g" % accuracy_log)

Logistic Regression Accuracy = 0.955091


## Tree

In [153]:
colname=mydf_new.columns

In [154]:
colname.remove('date')

In [157]:
colname.remove('failure')

In [158]:
colname.remove('model')

In [159]:
colname.remove('serial_number')

In [160]:
vectorAssembler_features = VectorAssembler(
    inputCols=colname, 
    outputCol="features")

In [161]:
splitted_data = mydf_new.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 45892843
Number of testing records : 10324465
Number of prediction records : 1146966


In [162]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="failure", featuresCol="features", numTrees=10)

In [163]:
pipeline_rf= Pipeline(stages=[modelcol, serial_number,
                               vectorAssembler_features, 
                               rf])

In [164]:
%%time
model_rf = pipeline_rf.fit(train_data)

CPU times: user 96.3 ms, sys: 15.3 ms, total: 112 ms
Wall time: 12min 45s


In [166]:
predictions_rf = model_rf.transform(test_data)
evaluatorRf = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("failure")
accuracy_rf = evaluatorRf.evaluate(predictions_rf)

print("Random Forest Accuracy = %g" % accuracy_rf)

Random Forest Accuracy = 0.823169


## GBModel

In [183]:
colname=mydf_new.columns

In [184]:
colname.remove('date')

In [185]:
colname.remove('failure')

In [186]:
colname.remove('model')

In [187]:
colname.remove('serial_number')

In [188]:
vectorAssembler_features = VectorAssembler(
    inputCols=colname, 
    outputCol="features")

In [189]:
splitted_data = mydf_new.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 45892843
Number of testing records : 10324465
Number of prediction records : 1146966


In [190]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=5, maxDepth=2, featuresCol="features",labelCol="failure", seed=1)

In [191]:
pipeline_gbt= Pipeline(stages=[vectorAssembler_features, 
                               gbt])

In [192]:
model_gbt = pipeline_gbt.fit(train_data)

In [193]:
predictions_gbt = model_gbt.transform(test_data)
evaluatorGbt = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction").setLabelCol("failure")
accuracy_gbt = evaluatorGbt.evaluate(predictions_gbt)

print("Gradient Boosting Tree Accuracy = %g" % accuracy_gbt)

Gradient Boosting Tree Accuracy = 0.74321


In [194]:
spark.stop()